# Estimation of Forrests using PNUTS

Instead of estimating the phylogenies, substitution and rate parameters for each language family separately, a hierarchical model will be defined, where the estimation can happen for several data sets at once. Thus a hyperprior on the substition paramters, the rates and the trees can be defined. Currently, this only works on the `Forrest` branch of the MCPhylo package.

**Although the whole library uses Multithreading. This estimation will be terribly slow. PNUTS for large trees is very slow.**

**This is just to demonstrate the setup. Multiple Chains etc. have to be used on a proper experiment.**

In [1]:
include("../src/MCPhylo.jl")
using .MCPhylo
using Random
Random.seed!(42);

┌ Warning: The Julia CUDA library is installed, but no CUDA device detected.
│          Computation is performed without CUDA functionality.
└ @ Main.MCPhylo C:\Programming\Julia_Tree\src\MCPhylo.jl:30


The different datasets are taken from Rama, et. al. (2018). (https://github.com/PhyloStar/AutoCogPhylo). First load the data, merge data with the tree and finally store necessary information in a dictionary.

In [2]:
mt_st, df_st = make_tree_with_data("data-st-64-110.paps.nex"); # load Sino-Tibetean
mt_ie, df_ie = make_tree_with_data("data-ie-42-208.paps.nex"); # load Indo-European
mt_aa, df_aa = make_tree_with_data("data-aa-58-200.paps.nex"); # load Austro-Asiatic
mt_an, df_an = make_tree_with_data("data-an-45-210.paps.nex"); # load Austornesian
mt_pn, df_pn = make_tree_with_data("data-pn-67-183.paps.nex"); # load Pama-Nyungan


function data_to_tree(mt, df)
    po = post_order(mt);
    for node in po
        node.data = df[:,:,node.num]
        node.scaler = zeros(1,size(node.data, 2))
    end
end

data_to_tree(mt_st, df_st)
data_to_tree(mt_ie, df_ie)
data_to_tree(mt_an, df_an)
data_to_tree(mt_aa, df_aa)
data_to_tree(mt_pn, df_pn)


my_data = Dict{Symbol, Any}(
  :mtree => [mt_st, mt_ie, mt_aa, mt_an, mt_pn],
  :df => [df_st, df_ie, df_aa, df_an, df_pn],
  :nnodes => [size(df_st)[3],size(df_ie)[3],size(df_aa)[3],size(df_an)[3],size(df_pn)[3]],
  :nbase => [size(df_st)[1],size(df_ie)[1],size(df_aa)[1],size(df_an)[1],size(df_pn)[1]],
  :nsites => [size(df_st)[2],size(df_ie)[2],size(df_aa)[2],size(df_an)[2],size(df_pn)[2]],
);


The next step is the definition of the model. Each dataset follows its own Phylogenetic Distribution. The likelihood of such a distribution object is calculated using Felsensteins algorithm. These distributions are defined by a tree, the frequency of the characters (e.g. `pi_ie`) and a vector of evolutionary rates. The equilibirum frequencies are all drawn from the same Dirichlet distribution with concentration vector `co`. The evoloutionary rates follow the discretized gamma distribution defined by Yang (1994) (https://link.springer.com/article/10.1007/BF00160154). Each dataset is modelled with its own set of rates, but the paramters of these different gamma distributions come from the same distribution.

In [3]:
# model setup
model =  Model(
    df_ie = Stochastic(3, (tree_ie, pi_ie,  rates) ->
                            PhyloDist(tree_ie, pi_ie[1], rates[1:4], my_data[:nbase][2], my_data[:nsites][2], my_data[:nnodes][2]), false, false),
    df_st = Stochastic(3, (tree_st, pi_st, rates) ->
                            PhyloDist(tree_st, pi_st[1], rates[5:8], my_data[:nbase][1], my_data[:nsites][1], my_data[:nnodes][1]), false, false),
    df_aa = Stochastic(3, (tree_aa, pi_aa, rates) ->
                            PhyloDist(tree_aa, pi_aa[1], rates[9:12], my_data[:nbase][3], my_data[:nsites][3], my_data[:nnodes][3]), false, false),
    df_an = Stochastic(3, (tree_an, pi_an, rates) ->
                            PhyloDist(tree_an, pi_an[1], rates[13:16], my_data[:nbase][4], my_data[:nsites][4], my_data[:nnodes][4]), false, false),
    df_pn = Stochastic(3, (tree_pn, pi_pn, rates) ->
                            PhyloDist(tree_pn, pi_pn[1], rates[17:20], my_data[:nbase][5], my_data[:nsites][5], my_data[:nnodes][5]), false, false),
    pi_ie = Stochastic(1, (co) -> Dirichlet(co)),
    pi_st = Stochastic(1, (co) -> Dirichlet(co)),
    pi_aa = Stochastic(1, (co) -> Dirichlet(co)),
    pi_an = Stochastic(1, (co) -> Dirichlet(co)),
    pi_pn = Stochastic(1, (co) -> Dirichlet(co)),
    co = Stochastic(1, () -> Gamma()),
    tree_ie = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true),
    tree_st = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true),
    tree_aa = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true),
    tree_an = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true),
    tree_pn = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true),
    rates = Logical(1, (αs, βs) -> vcat(discrete_gamma_rates.(αs, βs, 4)...),false),
    αs = Stochastic(1, () -> Gamma(), true),
    βs = Stochastic(1, () -> Gamma(), true)
     );


Next we need to set the initial values of the model. Since the trees are random, we will use these and all the other paramters are chosen randomly.

In [6]:
# intial model values
inits = [ Dict{Symbol, Union{Any, Real}}(
    :tree_ie => mt_ie,
    :tree_st => mt_st,
    :tree_pn => mt_pn,
    :tree_aa => mt_aa,
    :tree_an => mt_an,
    :pi_ie=> rand(Dirichlet(2, 1)),
    :pi_st=> rand(Dirichlet(2, 1)),
    :pi_an=> rand(Dirichlet(2, 1)),
    :pi_aa=> rand(Dirichlet(2, 1)),
    :pi_pn=> rand(Dirichlet(2, 1)),
    :df_ie => my_data[:df][2],
    :df_st => my_data[:df][1],
    :df_aa => my_data[:df][3],
    :df_an => my_data[:df][4],
    :df_pn => my_data[:df][5],
    :αs => rand(5),
    :βs => rand(5),
    :co => rand(2),
    ),
    ];

Next, select the apropriate samplers. The trees are sampled using the PNUTS algorithm. The other paramteres are sampled using an apropriate slice sampler.

In [7]:
scheme = [PNUTS(:tree_ie),
          PNUTS(:tree_st),
          PNUTS(:tree_an),
          PNUTS(:tree_aa),
          PNUTS(:tree_pn),
          SliceSimplex(:pi_ie),
          SliceSimplex(:pi_an),
          SliceSimplex(:pi_st),
          SliceSimplex(:pi_aa),
          SliceSimplex(:pi_pn),
          Slice(:co, 1.0, Univariate),
          Slice(:αs, 1.0, Univariate),
          Slice(:βs, 1.0, Univariate),
          ]

setsamplers!(model, scheme);

All that is left, is to start the sampler and store the results in the appropriate files.

In [ ]:
sim = mcmc(model, my_data, inits, 10, burnin=5,thin=1, chains=1, trees=true)

to_file(sim, "Forrest_")